In [3]:
from services.data_provider import DataLoader, feature_columns
from models import outcome_model
from db.interface import open_connection, close_connection

In [4]:
conn = open_connection()

# Optimize hyperparameters

In [ ]:
param_grid = {
    'max_depth': [3, 5, 8, 12, None],
    'min_samples_leaf': [10, 15, 20, 25, 30, 40, 50],
    'max_features': ["sqrt", "log2"],
    'n_estimators': [2000]
}

hyperparam_optimization_dl = DataLoader(feature_columns, "outcome")
best_params, score = run_grid_search(param_grid, hyperparam_optimization_dl)
best_params

In [ ]:
season_id = 53
data_loader = DataLoader(feature_columns, "outcome", filter_season=[season_id])
X, y = data_loader.get_dataset()

In [ ]:
model = get_model(best_params, X, y)
predictor = OutcomePredictor(model)
simulator = SeasonSimulator(season_id, data_loader, predictor, conn)
simulator.run()

In [ ]:
plot_simulation(simulator)

In [ ]:
df = pd.DataFrame(simulator.matches)
plot_probabilities_against_market(
    simulator.probabilities, 
    get_implied_probabilities(simulator.odds), "outcome_model")

In [ ]:
close_connection(conn)